In [ ]:
import findspark
findspark.init()
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("ECE552_FinalProject") \
    .config("spark.mongodb.input.uri", "mongodb://127.0.0.1/ECE552FinalProject.TrafficData") \
    .config("spark.mongodb.output.uri", "mongodb://127.0.0.1/ECE552FinalProject.TrafficData") \
    .getOrCreate()
sc = SparkContext.getOrCreate()
sc.setJobGroup("DA_Side", "DA_Side")

In [ ]:
%%time
pipeline = "[{ $group : { _id : '$Side', count:{$sum:1}}}, {$sort:{_id:1}}]"
df = spark.read.format("mongo").option("pipeline", pipeline).load()
df = df.groupby("_id").sum()
df = df.withColumnRenamed("_id","AccidentSide").withColumnRenamed("sum(count)","AccidentCount")
df = df.orderBy("AccidentSide")
print("Number of Documents: " + str(df.count()))
df.agg({'AccidentCount': 'sum'}).show()
df.show(49)

In [ ]:
df.write.format("mongo").option("collection", "AccidentCountBySide").mode("overwrite").save()